In [1]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import pandas as pd


In [2]:

def preprocess_text(text):
    text = text.lower()
    words = word_tokenize(text)
    stop_word = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_word]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    preprocessed_text = ' '.join(words)
    return preprocessed_text
    


In [3]:
import pandas as pd 
try:
    ds = pd.read_csv("C:/Users/CHETANYA UPADHYAY/OneDrive/Desktop/jupyter files/jupyter files.csv")
    print(ds.head(2))
except UnicodeDecodeError as err:
    print(f"Error : {err}")

                                                 URL  \
0  https://lawrato.com/indian-kanoon/ipc/section-140   
1  https://lawrato.com/indian-kanoon/ipc/section-127   

                                         Description  \
0  Description of IPC Section 140\nAccording to s...   
1  Description of IPC Section 127\nAccording to s...   

                                             Offense  \
0  Wearing the dress or carrying any token used b...   
1  Receiving property taken by war or depredation...   

                                Punishment  Cognizable      Bailable  \
0                 3 Months or Fine or Both  Cognizable      Bailable   
1  7 Years + Fine + forfeiture of property  Cognizable  Non-Bailable   

              Court  
0    Any Magistrate  
1  Court of Session  


In [4]:
ds.head(1)

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate


In [5]:
ds.isnull().sum

<bound method DataFrame.sum of        URL  Description  Offense  Punishment  Cognizable  Bailable  Court
0    False        False    False       False       False     False  False
1    False        False    False       False       False     False  False
2    False        False    False       False       False     False  False
3    False        False    False       False       False     False  False
4    False        False    False       False       False     False  False
..     ...          ...      ...         ...         ...       ...    ...
440  False        False    False       False       False     False  False
441  False        False    False       False       False     False  False
442  False        False    False       False       False     False  False
443  False        False    False       False       False     False  False
444  False         True     True        True        True      True   True

[445 rows x 7 columns]>

In [6]:
ds.fillna("Not Mentioned",inplace=True)

In [7]:
ds.isnull().sum()

URL            0
Description    0
Offense        0
Punishment     0
Cognizable     0
Bailable       0
Court          0
dtype: int64

In [8]:
ds['Combo'] = ds['Description']+ds['Offense']
ds.head(1)

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court,Combo
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate,Description of IPC Section 140\nAccording to s...


In [9]:
def preprocess_text(text):
    return text.lower()
ds['Combo'] = ds['Description'] + " " + ds['Offense'].astype(str)
ds['Combo'] = ds['Combo'].apply(preprocess_text)
ds.head(1)

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court,Combo
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate,description of ipc section 140\naccording to s...


In [10]:
new_ds = ds['Combo'].apply(lambda x: " ".join(x))
print(new_ds.head(1))


0    d e s c r i p t i o n   o f   i p c   s e c t ...
Name: Combo, dtype: object


In [11]:
new_ds = ds[['Description','Offense','Punishment','Cognizable','Bailable','Court','Combo']]
new_ds.head(1)

,Description,Offense,Punishment,Cognizable,Bailable,Court,Combo
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate,description of ipc section 140\naccording to s...


In [12]:
import pickle
with open('preprocess_data.pkl','wb') as file:
    pickle.dump(new_ds,file)
    new_ds=pickle.load(open('preprocess_data.pkl','rb'))


In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer,util
model=SentenceTransformer('paraphrase-MiniLM-L6-v2')

c:\Users\CHETANYA UPADHYAY\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [14]:
import torch
print(torch.__version__)

2.4.0+cpu


In [15]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [16]:
from tqdm.notebook import tqdm, trange
import importlib
sentence_transformers = importlib.import_module('sentence_transformers')
from sentence_transformers import SentenceTransformer

In [17]:
def suggest_sections(complaint,dataset,min_suggestions=5):
    preprocessed_complaint=preprocess_text(complaint)
    complaint_embedding=model.encode(preprocessed_complaint)
    
    try:
        section_embedding=model.encode([dataset['Combo']])
        similarities=util.pytorch_cos_sim(complaint_embedding,section_embedding)[0]
        similarity_threhold=0.2
        # Rest of your code
    except KeyError:
        print("Error: 'Combo' index is missing in the dataset")
        return []
    complaint_embedding=model.encode(preprocessed_complaint)
    section_embedding=model.encode(dataset['Combo'].tolist())
    similarities=util.pytorch_cos_sim(complaint_embedding,section_embedding)[0]
    similarity_threhold=0.2
    relevant_indices=[]
    while len(relevant_indices)<min_suggestions and similarity_threhold>0:
        relevant_indices=[i for i, sim in enumerate(similarities)if sim>similarity_threhold]
        similarity_threhold-=0.5 #st=st-0.5
        sorted_indices=sorted(relevant_indices,key=lambda i: similarities[i],reverse=True)
        suggestions=dataset.iloc[sorted_indices][['Description','Offense','Punishment','Cognizable','Bailable','Court','Combo']].to_dict(orient='records')
        return suggestions


In [18]:
complaint=input("Enter crime description")
suggested_sections=suggest_sections(complaint, new_ds)
if suggested_sections:
    print("Suggested Sections are : ")
    for suggestion in suggested_sections:
        print(f"Description : {suggestion['Description']}")
        print(f"Offense : {suggestion['Offense']}")
        print(f"Punishment : {suggestion['Punishment']}")
        print("___________________________________________________________\n")
else:
    print("No record found")

Suggested Sections are : 
Description : Description of IPC Section 302
According to section 302 of Indian penal code, Whoever commits murder shall be punished with death, or imprisonment for life, and shall also be liable to fine.


IPC 302 in Simple Words
Section 302 of the Indian Penal Code states that anyone who commits murder can be punished with the death penalty or life imprisonment, and may also be fined.

Cited by
Ramdeo Rai Yadav vs State Of Bihar
Mohammad Irfan vs State Of Karnataka
Prasad Pradhan vs The State Of Chhattisgarh
Nandu Singh vs The State Of Madhya Pradesh
Kala Singh Gurnam Singh vs The State Of Punjab
Offense : Murder
Punishment : Death or Imprisonment for Life + Fine
___________________________________________________________

Description : Description of IPC Section 300
According to section 300 of Indian penal code,
Except in the cases hereinafter excepted, culpable homicide is murder, if the act by which the death is caused is done with the intention of causin

In [19]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
